In [1]:
import numpy as np
import tensorflow as tf
import tensornets as nets

data_dir = '/home/taehoonlee/Data/imagenet/inputs'

In [2]:
def imagenet(data_dir, resize_wh, crop_wh, crops):
    return nets.utils.imagenet_val_generator(
        data_dir, batch_size=10 if crops==10 else 100,
        resize_wh=resize_wh,
        crop_locs=10 if crops==10 else 4,
        crop_wh=crop_wh)


def test(models_list, crops=1, verbose=False):
    batches1 = imagenet(data_dir, 256, 224, crops)
    batches2 = imagenet(data_dir, 341, 299, crops)
    batches3 = imagenet(data_dir, 378, 331, crops)
    inputs, models, shapes, params = [], [], [], []
    labels, preds_list = [], []
    if verbose:
        print("")

    with tf.Graph().as_default():
        for (_net, _shape, _gpu) in models_list:
            with tf.device("gpu:%d" % _gpu):
                _input = tf.placeholder(tf.float32, [None] + list(_shape))
                _model = _net(_input, is_training=False)
                _weights = _model.get_weights()
                inputs.append(_input)
                models.append(_model)
                shapes.append(_shape)
                params.append(sum([w.shape.num_elements() for w in _weights]))

        with tf.Session() as sess:
            nets.pretrained(models)
            while True:
                try:
                    batch1, label1 = batches1.next()
                    batch2, label2 = batches2.next()
                    batch3, label3 = batches3.next()
                except:
                    break
                preds = sess.run(models, dict((i, m.preprocess(batch1 if s[0] == 224 else batch2 if s[0] == 299 else batch3))
                                              for (i, m, s) in zip(inputs, models, shapes)))
                if crops > 1:
                    preds = [np.mean(pred.reshape(-1, crops, 1000), axis=1) for pred in preds]
                labels.append(label1)
                preds_list.append(preds)
                if verbose:
                    print('.'),
        labels = np.concatenate(labels)

    if verbose:
        print("")

    print("Crops: %d" % crops)
    print("Samples: %d" % len(labels))
    print("|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |")
    print("|------------------|-------|-------|--------|--------|-------|")

    for i in range(len(models)):
        preds = np.concatenate([np.argsort(pred[i], axis=1)[:, -5:]
                                for pred in preds_list], axis=0)
        actuals = labels[:preds.shape[0]]
        top1 = (actuals == preds[:, -1])
        top5 = [1 if actual in pred else 0
                for (actual, pred) in zip(actuals, preds)]
        err = lambda x: 100 * (1 - sum(x) / float(len(x)))
        print("| %16s | %5d | %5d | %2.3f | %2.3f | %.1fM |" %
              (models[i].aliases[0][:16],
               sum(top1), sum(top5),
               err(top1), err(top5),
               params[i] / 10e5))

### Summary:

- Best performed with cv2.INTER_LINEAR:
  * ResNet
  * SqueezeNet
- Best performed with cv2.INTER_CUBIC (Top-1, Top-5, 10-5 errors for the following models are improved by 0.078%, 0.08%, 0.096% on average over Image.BICUBIC):
  * ResNetv2
  * ResNeXt
  * WideResNet
  * Inception2-4
  * InceptionResNet2
  * NASNetAmobile
  * VGG
  * DenseNet
- Best performed with Image.BICUBIC:
  * Inception1
  * NASNetAlarge
  * MobileNet

### crops=1, cv2.INTER_CUBIC

In [3]:
test([(nets.ResNet50, (224, 224, 3), 0),
      (nets.ResNet101, (224, 224, 3), 0),
      (nets.ResNet152, (224, 224, 3), 0),
      (nets.ResNeXt50, (224, 224, 3), 0),
      (nets.ResNeXt101, (224, 224, 3), 1),
      (nets.ResNeXt101c64, (224, 224, 3), 1),
      (nets.WideResNet50, (224, 224, 3), 1),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|         resnet50 | 37437 | 46009 | 25.126 | 7.982 | 25.6M |
|        resnet101 | 38210 | 46393 | 23.580 | 7.214 | 44.7M |
|        resnet152 | 38302 | 46559 | 23.396 | 6.882 | 60.4M |
|     resnext50c32 | 38870 | 46905 | 22.260 | 6.190 | 25.1M |
|    resnext101c32 | 39365 | 47147 | 21.270 | 5.706 | 44.3M |
|    resnext101c64 | 39747 | 47296 | 20.506 | 5.408 | 83.7M |
|     wideresnet50 | 39009 | 46967 | 21.982 | 6.066 | 69.0M |


In [4]:
test([(nets.ResNet50v2, (299, 299, 3), 0),
      (nets.ResNet101v2, (299, 299, 3), 1),
      (nets.ResNet152v2, (299, 299, 3), 1),
      (nets.ResNet200v2, (224, 224, 3), 0),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       resnet50v2 | 37737 | 46374 | 24.526 | 7.252 | 25.6M |
|      resnet101v2 | 38442 | 46756 | 23.116 | 6.488 | 44.7M |
|      resnet152v2 | 38882 | 46960 | 22.236 | 6.080 | 60.4M |
|      resnet200v2 | 39143 | 47076 | 21.714 | 5.848 | 64.9M |


In [5]:
test([(nets.Inception1, (224, 224, 3), 0),
      (nets.Inception2, (224, 224, 3), 1),
      (nets.Inception3, (299, 299, 3), 0),
      (nets.Inception4, (299, 299, 3), 0),
      (nets.InceptionResNet2, (299, 299, 3), 1),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       inception1 | 33420 | 43838 | 33.160 | 12.324 | 7.0M |
|       inception2 | 36852 | 45865 | 26.296 | 8.270 | 11.2M |
|       inception3 | 38949 | 46860 | 22.102 | 6.280 | 23.9M |
|       inception4 | 40060 | 47489 | 19.880 | 5.022 | 42.7M |
| inceptionresnet2 | 40128 | 47626 | 19.744 | 4.748 | 55.9M |


In [6]:
test([(nets.NASNetAlarge, (331, 331, 3), 0)])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|     nasnetAlarge | 41249 | 48002 | 17.502 | 3.996 | 93.5M |


In [7]:
test([(nets.NASNetAmobile, (224, 224, 3), 0),
      (nets.VGG16, (224, 224, 3), 0),
      (nets.VGG19, (224, 224, 3), 1),
      (nets.SqueezeNet, (224, 224, 3), 1),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|    nasnetAmobile | 37183 | 45927 | 25.634 | 8.146 | 7.7M |
|            vgg16 | 35634 | 45025 | 28.732 | 9.950 | 138.4M |
|            vgg19 | 35628 | 44994 | 28.744 | 10.012 | 143.7M |
|       squeezenet | 27217 | 39020 | 45.566 | 21.960 | 1.2M |


In [8]:
test([(nets.DenseNet121, (224, 224, 3), 0),
      (nets.DenseNet169, (224, 224, 3), 0),
      (nets.DenseNet201, (224, 224, 3), 1),
      (nets.MobileNet25, (224, 224, 3), 0),
      (nets.MobileNet50, (224, 224, 3), 1),
      (nets.MobileNet75, (224, 224, 3), 1),
      (nets.MobileNet100, (224, 224, 3), 0),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|      densenet121 | 37260 | 45989 | 25.480 | 8.022 | 8.1M |
|      densenet169 | 38037 | 46554 | 23.926 | 6.892 | 14.3M |
|      densenet201 | 38532 | 46729 | 22.936 | 6.542 | 20.2M |
|      mobilenet25 | 25791 | 37896 | 48.418 | 24.208 | 0.5M |
|      mobilenet50 | 32146 | 42812 | 35.708 | 14.376 | 1.3M |
|      mobilenet75 | 34206 | 44121 | 31.588 | 11.758 | 2.6M |
|     mobilenet100 | 35212 | 44752 | 29.576 | 10.496 | 4.3M |


### crops=10, cv2.INTER_CUBIC

In [9]:
test([(nets.ResNet50, (224, 224, 3), 0),
      (nets.ResNet101, (224, 224, 3), 0),
      (nets.ResNet152, (224, 224, 3), 0),
      (nets.ResNeXt50, (224, 224, 3), 0),
      (nets.ResNeXt101, (224, 224, 3), 1),
      (nets.ResNeXt101c64, (224, 224, 3), 1),
      (nets.WideResNet50, (224, 224, 3), 1),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|         resnet50 | 38470 | 46579 | 23.060 | 6.842 | 25.6M |
|        resnet101 | 39032 | 46954 | 21.936 | 6.092 | 44.7M |
|        resnet152 | 39227 | 47046 | 21.546 | 5.908 | 60.4M |
|     resnext50c32 | 39746 | 47295 | 20.508 | 5.410 | 25.1M |
|    resnext101c32 | 40213 | 47579 | 19.574 | 4.842 | 44.3M |
|    resnext101c64 | 40514 | 47718 | 18.972 | 4.564 | 83.7M |
|     wideresnet50 | 39854 | 47442 | 20.292 | 5.116 | 69.0M |


In [10]:
test([(nets.ResNet50v2, (299, 299, 3), 0),
      (nets.ResNet101v2, (299, 299, 3), 1),
      (nets.ResNet152v2, (299, 299, 3), 1),
      (nets.ResNet200v2, (224, 224, 3), 0),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       resnet50v2 | 38835 | 46994 | 22.330 | 6.012 | 25.6M |
|      resnet101v2 | 39559 | 47385 | 20.882 | 5.230 | 44.7M |
|      resnet152v2 | 39849 | 47520 | 20.302 | 4.960 | 60.4M |
|      resnet200v2 | 40034 | 47585 | 19.932 | 4.830 | 64.9M |


In [11]:
test([(nets.Inception1, (224, 224, 3), 0),
      (nets.Inception2, (224, 224, 3), 1),
      (nets.Inception3, (299, 299, 3), 0),
      (nets.Inception4, (299, 299, 3), 0),
      (nets.InceptionResNet2, (299, 299, 3), 1),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       inception1 | 34888 | 44877 | 30.224 | 10.246 | 7.0M |
|       inception2 | 37991 | 46559 | 24.018 | 6.882 | 11.2M |
|       inception3 | 39963 | 47481 | 20.074 | 5.038 | 23.9M |
|       inception4 | 40846 | 47897 | 18.308 | 4.206 | 42.7M |
| inceptionresnet2 | 40882 | 48019 | 18.236 | 3.962 | 55.9M |


In [12]:
test([(nets.NASNetAlarge, (331, 331, 3), 0)], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|     nasnetAlarge | 41650 | 48294 | 16.700 | 3.412 | 93.5M |


In [13]:
test([(nets.NASNetAmobile, (224, 224, 3), 0),
      (nets.VGG16, (224, 224, 3), 0),
      (nets.VGG19, (224, 224, 3), 1),
      (nets.SqueezeNet, (224, 224, 3), 1),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|    nasnetAmobile | 38390 | 46621 | 23.220 | 6.758 | 7.7M |
|            vgg16 | 36623 | 45583 | 26.754 | 8.834 | 138.4M |
|            vgg19 | 36625 | 45613 | 26.750 | 8.774 | 143.7M |
|       squeezenet | 29638 | 40711 | 40.724 | 18.578 | 1.2M |


In [14]:
test([(nets.DenseNet121, (224, 224, 3), 0),
      (nets.DenseNet169, (224, 224, 3), 0),
      (nets.DenseNet201, (224, 224, 3), 1),
      (nets.MobileNet25, (224, 224, 3), 0),
      (nets.MobileNet50, (224, 224, 3), 1),
      (nets.MobileNet75, (224, 224, 3), 1),
      (nets.MobileNet100, (224, 224, 3), 0),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|      densenet121 | 38237 | 46579 | 23.526 | 6.842 | 8.1M |
|      densenet169 | 38976 | 46930 | 22.048 | 6.140 | 14.3M |
|      densenet201 | 39337 | 47138 | 21.326 | 5.724 | 20.2M |
|      mobilenet25 | 27743 | 39402 | 44.514 | 21.196 | 0.5M |
|      mobilenet50 | 33861 | 43910 | 32.278 | 12.180 | 1.3M |
|      mobilenet75 | 35767 | 45061 | 28.466 | 9.878 | 2.6M |
|     mobilenet100 | 36750 | 45613 | 26.500 | 8.774 | 4.3M |


### crops=1, cv2.INTER_LINEAR

In [15]:
test([(nets.ResNet50, (224, 224, 3), 0),
      (nets.ResNet101, (224, 224, 3), 0),
      (nets.ResNet152, (224, 224, 3), 0),
      (nets.ResNeXt50, (224, 224, 3), 0),
      (nets.ResNeXt101, (224, 224, 3), 1),
      (nets.ResNeXt101c64, (224, 224, 3), 1),
      (nets.WideResNet50, (224, 224, 3), 1),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|         resnet50 | 37563 | 46093 | 24.874 | 7.814 | 25.6M |
|        resnet101 | 38224 | 46443 | 23.552 | 7.114 | 44.7M |
|        resnet152 | 38341 | 46583 | 23.318 | 6.834 | 60.4M |
|     resnext50c32 | 38812 | 46847 | 22.376 | 6.306 | 25.1M |
|    resnext101c32 | 39349 | 47104 | 21.302 | 5.792 | 44.3M |
|    resnext101c64 | 39697 | 47293 | 20.606 | 5.414 | 83.7M |
|     wideresnet50 | 38933 | 46911 | 22.134 | 6.178 | 69.0M |


In [16]:
test([(nets.ResNet50v2, (299, 299, 3), 0),
      (nets.ResNet101v2, (299, 299, 3), 1),
      (nets.ResNet152v2, (299, 299, 3), 1),
      (nets.ResNet200v2, (224, 224, 3), 0),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       resnet50v2 | 37714 | 46346 | 24.572 | 7.308 | 25.6M |
|      resnet101v2 | 38427 | 46730 | 23.146 | 6.540 | 44.7M |
|      resnet152v2 | 38851 | 46930 | 22.298 | 6.140 | 60.4M |
|      resnet200v2 | 39121 | 47083 | 21.758 | 5.834 | 64.9M |


In [17]:
test([(nets.Inception1, (224, 224, 3), 0),
      (nets.Inception2, (224, 224, 3), 1),
      (nets.Inception3, (299, 299, 3), 0),
      (nets.Inception4, (299, 299, 3), 0),
      (nets.InceptionResNet2, (299, 299, 3), 1),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       inception1 | 33559 | 43961 | 32.882 | 12.078 | 7.0M |
|       inception2 | 36812 | 45794 | 26.376 | 8.412 | 11.2M |
|       inception3 | 38959 | 46902 | 22.082 | 6.196 | 23.9M |
|       inception4 | 40042 | 47462 | 19.916 | 5.076 | 42.7M |
| inceptionresnet2 | 40078 | 47600 | 19.844 | 4.800 | 55.9M |


In [18]:
test([(nets.NASNetAlarge, (331, 331, 3), 0)])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|     nasnetAlarge | 41248 | 47981 | 17.504 | 4.038 | 93.5M |


In [19]:
test([(nets.NASNetAmobile, (224, 224, 3), 0),
      (nets.VGG16, (224, 224, 3), 0),
      (nets.VGG19, (224, 224, 3), 1),
      (nets.SqueezeNet, (224, 224, 3), 1),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|    nasnetAmobile | 37199 | 45914 | 25.602 | 8.172 | 7.7M |
|            vgg16 | 35481 | 44940 | 29.038 | 10.120 | 138.4M |
|            vgg19 | 35528 | 44917 | 28.944 | 10.166 | 143.7M |
|       squeezenet | 27680 | 39342 | 44.640 | 21.316 | 1.2M |


In [20]:
test([(nets.DenseNet121, (224, 224, 3), 0),
      (nets.DenseNet169, (224, 224, 3), 0),
      (nets.DenseNet201, (224, 224, 3), 1),
      (nets.MobileNet25, (224, 224, 3), 0),
      (nets.MobileNet50, (224, 224, 3), 1),
      (nets.MobileNet75, (224, 224, 3), 1),
      (nets.MobileNet100, (224, 224, 3), 0),])

Crops: 1
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|      densenet121 | 37294 | 45952 | 25.412 | 8.096 | 8.1M |
|      densenet169 | 38023 | 46455 | 23.954 | 7.090 | 14.3M |
|      densenet201 | 38591 | 46738 | 22.818 | 6.524 | 20.2M |
|      mobilenet25 | 25843 | 37948 | 48.314 | 24.104 | 0.5M |
|      mobilenet50 | 32214 | 42844 | 35.572 | 14.312 | 1.3M |
|      mobilenet75 | 34178 | 44128 | 31.644 | 11.744 | 2.6M |
|     mobilenet100 | 35263 | 44781 | 29.474 | 10.438 | 4.3M |


### crops=10, cv2.INTER_LINEAR

In [21]:
test([(nets.ResNet50, (224, 224, 3), 0),
      (nets.ResNet101, (224, 224, 3), 0),
      (nets.ResNet152, (224, 224, 3), 0),
      (nets.ResNeXt50, (224, 224, 3), 0),
      (nets.ResNeXt101, (224, 224, 3), 1),
      (nets.ResNeXt101c64, (224, 224, 3), 1),
      (nets.WideResNet50, (224, 224, 3), 1),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|         resnet50 | 38563 | 46640 | 22.874 | 6.720 | 25.6M |
|        resnet101 | 39116 | 46964 | 21.768 | 6.072 | 44.7M |
|        resnet152 | 39272 | 47077 | 21.456 | 5.846 | 60.4M |
|     resnext50c32 | 39701 | 47297 | 20.598 | 5.406 | 25.1M |
|    resnext101c32 | 40155 | 47543 | 19.690 | 4.914 | 44.3M |
|    resnext101c64 | 40496 | 47672 | 19.008 | 4.656 | 83.7M |
|     wideresnet50 | 39811 | 47425 | 20.378 | 5.150 | 69.0M |


In [22]:
test([(nets.ResNet50v2, (299, 299, 3), 0),
      (nets.ResNet101v2, (299, 299, 3), 1),
      (nets.ResNet152v2, (299, 299, 3), 1),
      (nets.ResNet200v2, (224, 224, 3), 0),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       resnet50v2 | 38803 | 46971 | 22.394 | 6.058 | 25.6M |
|      resnet101v2 | 39528 | 47383 | 20.944 | 5.234 | 44.7M |
|      resnet152v2 | 39814 | 47513 | 20.372 | 4.974 | 60.4M |
|      resnet200v2 | 40048 | 47573 | 19.904 | 4.854 | 64.9M |


In [23]:
test([(nets.Inception1, (224, 224, 3), 0),
      (nets.Inception2, (224, 224, 3), 1),
      (nets.Inception3, (299, 299, 3), 0),
      (nets.Inception4, (299, 299, 3), 0),
      (nets.InceptionResNet2, (299, 299, 3), 1),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|       inception1 | 35097 | 44976 | 29.806 | 10.048 | 7.0M |
|       inception2 | 37914 | 46468 | 24.172 | 7.064 | 11.2M |
|       inception3 | 39958 | 47477 | 20.084 | 5.046 | 23.9M |
|       inception4 | 40814 | 47878 | 18.372 | 4.244 | 42.7M |
| inceptionresnet2 | 40830 | 48015 | 18.340 | 3.970 | 55.9M |


In [24]:
test([(nets.NASNetAlarge, (331, 331, 3), 0)], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|     nasnetAlarge | 41644 | 48283 | 16.712 | 3.434 | 93.5M |


In [25]:
test([(nets.NASNetAmobile, (224, 224, 3), 0),
      (nets.VGG16, (224, 224, 3), 0),
      (nets.VGG19, (224, 224, 3), 1),
      (nets.SqueezeNet, (224, 224, 3), 1),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|    nasnetAmobile | 38352 | 46580 | 23.296 | 6.840 | 7.7M |
|            vgg16 | 36438 | 45493 | 27.124 | 9.014 | 138.4M |
|            vgg19 | 36471 | 45485 | 27.058 | 9.030 | 143.7M |
|       squeezenet | 30085 | 41014 | 39.830 | 17.972 | 1.2M |


In [26]:
test([(nets.DenseNet121, (224, 224, 3), 0),
      (nets.DenseNet169, (224, 224, 3), 0),
      (nets.DenseNet201, (224, 224, 3), 1),
      (nets.MobileNet25, (224, 224, 3), 0),
      (nets.MobileNet50, (224, 224, 3), 1),
      (nets.MobileNet75, (224, 224, 3), 1),
      (nets.MobileNet100, (224, 224, 3), 0),], 10)

Crops: 10
Samples: 50000
|                  | Top-1 | Top-5 | Top-1  | Top-5  | Size  |
|------------------|-------|-------|--------|--------|-------|
|      densenet121 | 38170 | 46547 | 23.660 | 6.906 | 8.1M |
|      densenet169 | 38917 | 46920 | 22.166 | 6.160 | 14.3M |
|      densenet201 | 39350 | 47144 | 21.300 | 5.712 | 20.2M |
|      mobilenet25 | 27815 | 39476 | 44.370 | 21.048 | 0.5M |
|      mobilenet50 | 33900 | 43952 | 32.200 | 12.096 | 1.3M |
|      mobilenet75 | 35798 | 45117 | 28.404 | 9.766 | 2.6M |
|     mobilenet100 | 36749 | 45612 | 26.502 | 8.776 | 4.3M |
